In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import pyautogui

In [50]:
# Move to mouse 

pyautogui.moveTo(1500,400,duration=1)

pyautogui.FAILSAFE=False

driver_path = "/usr/local/bin/chromedriver"

main_url = "https://www.hepsiburada.com/"

# Create an empty DataFrame for user reviews and products

reviews_df = pd.DataFrame(columns=['user reviews', 'rating', 'product url'])

products_df = pd.DataFrame(columns=['product url', 'product name', 'img url','brand name','category name'])

service = ChromeService(executable_path=driver_path)

driver = webdriver.Chrome(service=service)

driver.get(main_url)

for j in range(0,18,2):

    # Get the category name

    action = ActionChains(driver)

    navbar_element = driver.find_element(By.CLASS_NAME,'sf-MenuItems-WulWXvlfIAwNiOUGY7FP') 

    driver.execute_script("arguments[0].scrollIntoView();", navbar_element)

    action.move_to_element(navbar_element).perform()

    time.sleep(4)
    
    category_tag = driver.find_elements(By.CLASS_NAME,'sf-ChildMenuItems-a4G0z3YJJWkQs7qKKuuj')[j]

    category_name = category_tag.text

    category_tag.click()

    time.sleep(5)

    # Sort the products from those with the most comments to fewest 

    order_button = driver.find_elements(By.CSS_SELECTOR,"div[data-test-id='dropdown-container']")[1]

    driver.execute_script("arguments[0].scrollIntoView();", order_button)

    order_button.click()

    time.sleep(5)

    sort_by_reviews = driver.find_elements(By.CLASS_NAME,'horizontalSortingBar-PkoDOH7UsCwBrQaQx9bn')

    sort_by_reviews[3].click()

    driver.refresh()

    # Find <li> tags containing the list of products

    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME,"productListContent-zAP0Y5msy8OHn5z7T_K_"))
    )

    li_tags = driver.find_elements(By.CLASS_NAME,"productListContent-zAP0Y5msy8OHn5z7T_K_")

    # Loop through each product

    for li in li_tags:
        product_url = None
        product_name = None
        img_url = None
        user_review = None
        rating = None
        brand_name = None
        
        # Get the product URL
        product_url = li.find_element(By.TAG_NAME,"a").get_attribute("href")

        # Open a new window, Switch to the new window
        driver.execute_script("window.open('');")

        driver.switch_to.window(driver.window_handles[-1]) 

        driver.get(product_url)

        driver.set_window_size(1200, driver.get_window_size()["height"])

        # Get the product name
        product_name = driver.find_element(By.ID,"product-name").text

        # Get the brand name 
        brand_name = driver.find_element(By.XPATH,"/html/body/div[2]/main/div[3]/section[1]/div[3]/div/div[4]/div[1]/span/a").text

        # Get the product image URL
        img_url = driver.find_element(By.CSS_SELECTOR,"img[class='product-image']").get_attribute("src")

        # Add products to the DataFrame
        products_df.loc[len(products_df)]=[product_url,product_name,img_url,brand_name,category_name]

        # Scroll down the page, Wait for user reviews to load
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        driver.find_element(By.XPATH,'//*[@id="productReviewsTab"]').click()

        # Process each user review
        try:
            all_reviews_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//*[@id='hermes-voltran-comments']/div[2]/div[1]/a"))).click()

            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

            try:
                last_reviews = driver.find_elements(By.XPATH,'//*[@id="hermes-voltran-comments"]/div[6]/div[3]/div[2]/div/ul/li')[-1].text
                
            except IndexError:
                last_reviews = 1

            driver.set_window_size(400, driver.get_window_size()["height"])

            #for i in range(int(last_reviews)):
            i = 0
            
            while i < int(last_reviews):

                next_page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
                    (By.XPATH,'//*[@id="hermes-voltran-comments"]/div[6]/div[3]/div[2]/div/div/div/div[3]')))
                
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)

                try:

                    all_reviews_div = WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.CLASS_NAME,"hermes-ReviewCard-module-BJtQZy5Ub3goN_D0yNOP")))

                    WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[class='star']")))

                    WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span[itemprop='description']")))

                except:

                    pass
                
                counter = 0

                for review in all_reviews_div:

                    try:
                        rating = len(review.find_elements(By.CSS_SELECTOR, "div[class='star']"))
                    except:
                        rating = None

                    try:
                        user_review = review.find_element(By.CSS_SELECTOR, "span[itemprop='description']").text

                    except:
                        #user_review = None

                        # Counter to check review without text

                        counter += 1

                        if counter == 3:
                            i = int(last_reviews)
                            break

                    # Add products to the DataFrame   
                    reviews_df.loc[len(reviews_df)] = [user_review,rating,product_url]

            
                next_page_button.click()

                i+=1
                
            driver.close()

            # Switch back to the main window
            driver.switch_to.window(driver.window_handles[0])

        except TimeoutException:

            driver.close()
            
            # Switch back to the main window
            driver.switch_to.window(driver.window_handles[0])
            continue

    driver.set_window_size(1200, driver.get_window_size()["height"])

driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.184)
Stacktrace:
0   chromedriver                        0x0000000104a127dc chromedriver + 4040668
1   chromedriver                        0x0000000104a0a9e0 chromedriver + 4008416
2   chromedriver                        0x000000010467d870 chromedriver + 284784
3   chromedriver                        0x0000000104658064 chromedriver + 131172
4   chromedriver                        0x00000001046e72b0 chromedriver + 717488
5   chromedriver                        0x00000001046fa75c chromedriver + 796508
6   chromedriver                        0x00000001046b574c chromedriver + 513868
7   chromedriver                        0x00000001046b6044 chromedriver + 516164
8   chromedriver                        0x00000001049d7a04 chromedriver + 3799556
9   chromedriver                        0x00000001049dbee4 chromedriver + 3817188
10  chromedriver  

In [49]:
reviews_df

,user reviews,rating,product url
0,Ürünün 180 günlük sınırlı bir süre ile gelmesi...,3,https://www.hepsiburada.com/gakademi-goruntulu...
1,Pek bi işe yaradığını düşünmüyorum keşke bunun...,5,https://www.hepsiburada.com/gakademi-goruntulu...
2,"Güzel bir hediye ancak, keşke kargolamak yerin...",5,https://www.hepsiburada.com/gakademi-goruntulu...
3,"Hepsiburada ürünümü göndermeyip, ayrıca müşter...",5,https://www.hepsiburada.com/gakademi-goruntulu...
4,Amerikan English Kursu 8 kuru bu fiyata veriyo...,5,https://www.hepsiburada.com/gakademi-goruntulu...
...,...,...,...
155,"Ürün güzel laptop aldım hediye geldi. fakat, i...",5,https://www.hepsiburada.com/gakademi-goruntulu...
156,Ürün sorunsuz ulaştı. Kargolama hızlı ve özenl...,5,https://www.hepsiburada.com/gakademi-goruntulu...
157,Hızlı kargo için teşekkürler.Sınırsız süre boy...,5,https://www.hepsiburada.com/gakademi-goruntulu...
158,orta son ya da liseye giden çocuğunuz varsa gü...,5,https://www.hepsiburada.com/gakademi-goruntulu...


In [44]:
products_df

,product url,product name,img url,brand name,category name
0,https://www.hepsiburada.com/gakademi-goruntulu...,GAkademi Görüntülü Online Eğitim Platformu,https://productimages.hepsiburada.net/s/43/550...,Görüntülü Akademi,Bilgisayar/Tablet
1,https://www.hepsiburada.com/xiaomi-mi-wifi-pro...,Xiaomi Mi Wifi Pro Sinyal Yakınlaştırıcı - Güç...,https://productimages.hepsiburada.net/s/36/550...,Xiaomi,Bilgisayar/Tablet
2,https://www.hepsiburada.com/sandisk-ssd-plus-2...,Sandisk SSD Plus 240GB 530MB-440MB/s Sata 3 2....,https://productimages.hepsiburada.net/s/9/550/...,Sandisk,Bilgisayar/Tablet


In [45]:
a=products_df.iloc[1,0]

In [46]:

reviews_df[reviews_df["product url"]==a]

,user reviews,rating,product url
2918,modemim salonda bulunuyor. evim koridorlu ve u...,5,https://www.hepsiburada.com/xiaomi-mi-wifi-pro...
2919,"Üründen önce satıcı yorumum: ""voltronx"" isimli...",5,https://www.hepsiburada.com/xiaomi-mi-wifi-pro...
2920,"Arka odalarda internet kullanamıyorduk, ürünün...",5,https://www.hepsiburada.com/xiaomi-mi-wifi-pro...
2921,Ürünü daha önce kullandığım için şimdide kızım...,5,https://www.hepsiburada.com/xiaomi-mi-wifi-pro...
2922,"Kolay kurulumu mevcut, çekim gücüde iyi, kopma...",5,https://www.hepsiburada.com/xiaomi-mi-wifi-pro...
2923,Ürün muhteşem çekmeyen modem yeniden doğdu,5,https://www.hepsiburada.com/xiaomi-mi-wifi-pro...
2924,Çok güzel aldım kullanıyorum şimdilik çok iyi ...,5,https://www.hepsiburada.com/xiaomi-mi-wifi-pro...
2925,Ürün çok hoşuma gitti çekim gücü süper teşekkü...,5,https://www.hepsiburada.com/xiaomi-mi-wifi-pro...
2926,Sağlam mal,5,https://www.hepsiburada.com/xiaomi-mi-wifi-pro...
2927,Gercekten harika bir urun hizli kargo icin tes...,5,https://www.hepsiburada.com/xiaomi-mi-wifi-pro...


In [23]:
reviews_df[reviews_df[["user reviews","rating"]].isna()]

,user reviews,rating,product url
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
7105,None,None,NaN
7106,None,None,NaN
7107,None,None,NaN
7108,None,None,NaN


In [17]:
len(reviews_df["user reviews"].unique())

2320

In [18]:
len(reviews_df[reviews_df["user reviews"].isna()])

4196

In [61]:
products_df.to_csv('products_df.csv',index=False,encoding="utf-8")

In [62]:
reviews_df.to_csv('reviews_df.csv',index=False,encoding="utf-8")